In [1]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [2]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

In [5]:
from posarmctools.ekfnavtools import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
idx_timeStamp

0

## Load navigation data SbgLogEkfNav

In [4]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_21-22_SBG/dataLogger"

logEkfNav  = prefix + "/session_0003/2017_09_22/13h00/" + "sbgLogEkfNav.dat"
logUtcData = prefix + "/session_0003/2017_09_22/13h00/" + "sbgLogUtcData.dat"

Load logEkfNav data 13h00

In [5]:
# logEkfNav
timeStamps = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_timeStamp )
Long = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Long )
Lat = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Lat )
Alt = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Alt )
vN, vE, vD = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [6]:
Vel = (vN**2 + vE**2 + vD**2)**0.5

### Plot navigation data jointly with track references

In [12]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

plt.title("Navigation data and track references 2017_09_22 13h00")

### Check the existence of record files

In [13]:
firstRecord = 960
lastRecord = 3372
aux = np.zeros(3372)
fileList = []
dataDir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_22_15_21_51"
for k in range(firstRecord, lastRecord, blocksPerFile):
    filename = dataDir + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)
idxFileList = np.arange( len( fileList ) ) * blocksPerFile + firstRecord

In [14]:
plt.plot(idxFileList, fileList, 'o-')

In [15]:
# times recorded by Windows XP
t_0    = (15, 21, 51)

record0Start = 960
record0Stop = 1866

record1Start = 1944
record1Stop = 2082

record2Start = 2190
record2Stop = 3372

Look for the time data corresponding to the records

In [16]:
hourOffset = 2

#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8
utc_h, utc_m, utc_s, utc_nano = np.loadtxt( logUtcData, skiprows = 1, 
                                           usecols = (idx_h, idx_m, idx_s, idx_nano), unpack=True )
utc_seconds = (utc_h + hourOffset) * 3600 + utc_m * 60 + utc_s + utc_nano * 1e-9

In [18]:
# compute time of day using the date of block 0 and a computed delta

record0_tStart = getTimeOfDay( getSeconds( t_0 ) + record0Start / blocksPerFile * T_files )
record0_tStop = getTimeOfDay( getSeconds( t_0 ) + record0Stop / blocksPerFile * T_files )

record1_tStart = getTimeOfDay( getSeconds( t_0 ) + record1Start / blocksPerFile * T_files )
record1_tStop = getTimeOfDay( getSeconds( t_0 ) + record1Stop / blocksPerFile * T_files )

record2_tStart = getTimeOfDay( getSeconds( t_0 ) + record2Start / blocksPerFile * T_files )
record2_tStop = getTimeOfDay( getSeconds( t_0 ) + record2Stop / blocksPerFile * T_files )

In [20]:
record0_idx = getIndices (record0_tStart, record0_tStop)
record1_idx = getIndices (record1_tStart, record1_tStop)
record2_idx = getIndices (record2_tStart, record2_tStop)

### Plot navigation data and record periods

In [25]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )
plotRunaway( ax )

addOnPlot( ax, record0_idx, 'yellow', 'record 0' )
addOnPlot( ax, record1_idx, 'red', 'record 1' )
addOnPlot( ax, record2_idx, 'magenta', 'record 2' )

plt.title("Navigation data, track references, record locations\nFlight 2017_09_22 13h00")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

## Track 1

In [26]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long, '.')
ax1.plot(np.ones(Long.shape) * track1_Long_0)
ax1.plot(np.ones(Long.shape) * track1_Long_1)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat, '.')
ax2.plot(np.ones(Lat.shape) * track1_Lat_0)
ax2.plot(np.ones(Lat.shape) * track1_Lat_1)

### Track 1 reference / Flight over 1

In [ ]:
idx_allTr3_fo1 = (515398, 527835)

In [31]:
# 960
start = 960
stop = start + 6 * 15
spot1_0 = getTimeOfDay( getSeconds( t_0 ) + start / blocksPerFile * T_files )
spot1_1 = getTimeOfDay( getSeconds( t_0 ) + stop / blocksPerFile * T_files )
idx_spot1_0 = np.amax( np.where( utc_seconds <= getSeconds(spot1_0) ) )
idx_spot1_1 = np.amax( np.where( utc_seconds <= getSeconds(spot1_1) ) )
idx_spot1 = (idx_spot1_0, idx_spot1_1)
velocity_spot1_0_spot1_1 = np.average( Vel[idx_spot1_0:idx_spot1_1] )
altitude_spot1_0_spot1_1 = np.average( Alt[idx_spot1_0:idx_spot1_1] )

In [34]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )
plotRunaway( ax )

addOnPlot( ax, idx_spot1, 'red', 'selection' )
addSpot( ax, idx_spot1_0, 'white' )
addSpot( ax, idx_spot1_1, 'white' )

plt.title("Navigation data and track references 2017_09_22 13h00\nTrack 1 Flight over 1")
plt.legend()
plt.grid()

# Data selection

## Track 1

### Runaway Flight over 1

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_tr1_fo1 )

### Runaway Flight over 2

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_tr1_fo2 )

### Track 1 Flight over 1

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_allTr1_fo1 )

### Track 1 Flight over 2

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_allTr1_fo2 )

### Track 3 Flight over 1

In [ ]:
getSelection(utc_seconds, Alt, Vel, (idx_spot3_0, idx_spot3_1) )